In [1]:
from kedro.framework.context import load_context
context = load_context("/home/lukas/pjs/operativelogik/op-logik")
catalog = context.catalog

The default value of auto_mkdir=True has been deprecated and will be changed to auto_mkdir=False by default in a future release.


In [10]:
filfeaturedict = context.catalog.load('filfeaturedict')
filialdata = context.catalog.load('filialdata')

2020-12-03 17:21:46,163 - kedro.io.data_catalog - INFO - Loading data from `filfeaturedict` (PickleDataSet)...
2020-12-03 17:21:46,759 - kedro.io.data_catalog - INFO - Loading data from `filialdata` (CSVDataSet)...


In [18]:
fdict = context.catalog.load('fdict')
weatherdict = context.catalog.load('weatherdict')
parameters = context.catalog.load('parameters')
sales_lag = context.catalog.load('sales_lag')

2020-12-26 15:06:51,745 - kedro.io.data_catalog - INFO - Loading data from `fdict` (PickleDataSet)...
2020-12-26 15:06:57,065 - kedro.io.data_catalog - INFO - Loading data from `weatherdict` (PickleDataSet)...
2020-12-26 15:06:57,195 - kedro.io.data_catalog - INFO - Loading data from `parameters` (MemoryDataSet)...
2020-12-26 15:06:57,242 - kedro.io.data_catalog - INFO - Loading data from `sales_lag` (CSVDataSet)...


In [8]:
pred_dates = context.catalog.load('pred_dates')

2020-12-03 17:21:34,274 - kedro.io.data_catalog - INFO - Loading data from `pred_dates` (PickleDataSet)...


`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [11]:
filfeaturedict['2']['101']

,amount,date,discount,id,prediction,wochentag,monat,datecode,lag_7,lag_14,lag_21,lag_28,sunhour,weathercodes,feels_like,heat_index
0,NaN,2019-04-23,8,446801,NaN,1,4,2,95,90,57,90,14.1,116.0,14.0,15.0
1,NaN,2019-04-24,8,447201,NaN,2,4,2,68,72,89,77,14.2,116.0,15.0,17.0
2,NaN,2019-04-25,0,447601,NaN,3,4,2,104,68,129,88,14.2,116.0,14.0,15.0
3,NaN,2019-04-26,0,448001,NaN,4,4,2,0,114,108,62,8.9,302.0,6.0,9.0
4,NaN,2019-04-27,0,448401,NaN,5,4,2,138,142,123,348,11.1,353.0,6.0,9.0


In [78]:
basedict = context.catalog.load('basedict')

2020-11-27 07:56:17,645 - kedro.io.data_catalog - INFO - Loading data from `basedict` (PickleDataSet)...


In [16]:
postframe = context.catalog.load('postframe') 

2020-12-02 13:04:25,205 - kedro.io.data_catalog - INFO - Loading data from `postframe` (CSVDataSet)...


In [26]:
import json

In [36]:
postframe.groupby('artnr')['prediction'].mean().iloc[0]

202.56410256410257

In [4]:
import requests
import json

In [14]:
# json.dumps(list(postframe.groupby('artnr')['prediction'].mean().to_json()),indent=2)

In [195]:
lag = 7
lagbase = None
filiale = '2'
article = '101'
pred_duration = 5
start_date= '2019-04-23'
lags_used = [7, 14, 21, 28]
pred_dates = context.catalog.load('pred_dates')
api_length = 400
server_link = 'http://35.234.103.119/api'

2020-11-27 09:38:54,313 - kedro.io.data_catalog - INFO - Loading data from `pred_dates` (PickleDataSet)...


In [205]:
start = str(min(pred_dates) - datetime.timedelta(max(lags_used)))[:-9]
end = str(max(pred_dates) - datetime.timedelta(min(lags_used)))[:-9] # adapted ende
URL = server_link + '/sales/amount?dateFrom=' + start + '&dateTo=' + end + '&pageSize=' + str(api_length) + '&page='
print(URL)

http://35.234.103.119/api/sales/amount?dateFrom=2019-03-26&dateTo=2019-04-20&pageSize=400&page=


`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [211]:
import requests

In [3]:
URL = "http://35.234.103.119/api/sales/amount?dateFrom=2019-03-26&dateTo=2019-04-20&pageSize=10400"

In [276]:
start = str(min(pred_dates) - datetime.timedelta(max(lags_used)))[:-9]
end = str(max(pred_dates) - datetime.timedelta(min(lags_used)))[:-9] # adapted ende

In [282]:
(pd.to_datetime(end) - pd.to_datetime(start)).days 

25

In [4]:
raw = requests.get(url = URL)

In [5]:
df = pd.DataFrame(raw.json())

In [9]:
df.id.nunique()

10400

In [6]:
len(pd.Series(raw.json()).apply(lambda x: x['id']).unique())

10400

In [258]:
pd.Series(reqs).apply(lambda x: x['id']).unique()

array([435601, 435602, 435603, ..., 445721, 445722, 445723])

In [265]:
base = "http://35.234.103.119/api/sales/amount?dateFrom=2019-03-26&dateTo=2019-04-20&pageSize=400&page="
reqs = []
for i in pages:
    URL = base + str(i)
    rare = requests.get(url = URL)
    reqs += rare.json()
new = pd.concat(list(pd.Series(reqs).apply(lambda x: pd.json_normalize(x))))

In [ ]:
for filiale in sales_lag['filnr'].unique():
    for article in sales_lag['artnr'].unique():
        print(filiale, article, ' ', len(new[(new['filnr']==int(filiale))&(new['artnr']==int(article))]))

In [200]:
import nest_asyncio
import datetime
import asyncio
import aiohttp
nest_asyncio.apply()
def laggetter(lags_used, pred_dates, api_length, server_link):
    start = str(min(pred_dates) - datetime.timedelta(max(lags_used)))[:-9]
    end = str(max(pred_dates) - datetime.timedelta(min(lags_used)))[:-9] # adapted ende
    URL = server_link + '/sales/amount?dateFrom=' + start + '&dateTo=' + end + '&pageSize=' + str(api_length) + '&page='
    pages = range(1, max(lags_used)+1)
    list_df = func(URL, pages)
    df = pd.concat(list(pd.Series(list_df).apply(lambda x: pd.json_normalize(x))))
    df = df.iloc[pd.to_datetime(df.date).values.argsort()] # return df sorted by date after asynchronous calls
    return df
async def fetch(session, url):
    async with session.get(url) as response:
        json_response = await response.json()
        return json_response

async def main(URL, pages):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, (URL+str(i))) for i in pages]
        responses = await asyncio.gather(*tasks)
        return responses

def func(URL, pages):
    responses = asyncio.run(main(URL, pages))
    return responses

In [201]:
df = laggetter(lags_used, pred_dates, api_length, server_link)

http://35.234.103.119/api/sales/amount?dateFrom=2019-03-26&dateTo=2019-04-20&pageSize=400&page=


In [ ]:
http://35.234.103.119/api/sales/amount?dateFrom=2019-03-26&dateTo=2019-04-20&pageSize=400&page=


In [269]:
pd.DataFrame.from_dict(reqs).drop_duplicates('id').id.max()

446000

In [238]:
df[df['id']==436770]

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


,amount,artnr,date,filnr,id
369,47,626,2019-03-28,49,436770
137,47,626,2019-03-28,49,436770
137,47,626,2019-03-28,49,436770


In [67]:
start = str(min(pred_dates) - datetime.timedelta(max(lags_used)))[:-9]

In [17]:
total = len(requests.get(url= parameters['server_link'] + '/sales/unique/id').json())

In [20]:
n_arts = len(requests.get(url= parameters['server_link'] + '/sales/unique/artnr').json())
n_fils = len(requests.get(url= parameters['server_link'] + '/sales/unique/filnr').json())
api_length = n_arts * n_fils

In [153]:
for lag in parameters['lags_used']:
    print(lag-(parameters['lags_used'][0]-pred_duration))

5
12
19
26


In [140]:
start = lag-(lag-pred_duration)
# faster: apply call on df to return "lagdf" for merging?
sales_lag[(sales_lag['filnr']==int(filiale))&(sales_lag['artnr']==int(article))].iloc[-start:lagbase]['amount'].values

array([  0, 138, 138, 653, 273])

In [149]:
sales_lag[(sales_lag['filnr']==int(filiale))&(sales_lag['artnr']==int(article))].iloc[-start:lagbase].amount.values

array([  0, 138, 138, 653, 273])

In [112]:
sales_lag[(sales_lag['filnr']==int(filiale))&(sales_lag['artnr']==int(article))].iloc[-lag:None]#['amount'].values

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


,amount,artnr,date,filnr,id
7824,306,101,2019-04-14,2,443201
8265,61,101,2019-04-15,2,443601
8595,95,101,2019-04-16,2,444001
9069,68,101,2019-04-17,2,444401
9435,104,101,2019-04-18,2,444801
9905,0,101,2019-04-19,2,445201
10269,138,101,2019-04-20,2,445601


In [93]:
sales_lag['date'].unique()

array(['2019-03-26', '2019-03-27', '2019-03-28', '2019-03-29',
       '2019-03-30', '2019-03-31', '2019-04-01', '2019-04-02',
       '2019-04-03', '2019-04-04', '2019-04-05', '2019-04-06',
       '2019-04-07', '2019-04-08', '2019-04-09', '2019-04-10',
       '2019-04-11', '2019-04-12', '2019-04-13', '2019-04-14',
       '2019-04-15', '2019-04-16', '2019-04-17', '2019-04-18',
       '2019-04-19', '2019-04-20'], dtype=object)

In [30]:
'/sales?dateFrom=' + start + '&dateTo=' + ende + '&pageSize=' + str(api_length) + '&page='

'/sales?dateFrom=2019-03-29&dateTo=2019-04-23&pageSize=400&page='

In [74]:
# sales_lag[(sales_lag['filnr']==int(filiale))&(sales_lag['artnr']==int(article))]

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [9]:
pred_dates = context.catalog.load('pred_dates')

2020-11-26 19:18:26,236 - kedro.io.data_catalog - INFO - Loading data from `pred_dates` (PickleDataSet)...


`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [8]:
salesdata = context.catalog.load('salesdata')

2020-12-25 19:44:14,868 - kedro.io.data_catalog - INFO - Loading data from `salesdata` (CSVDataSet)...


In [5]:
salesdata.head()

,amount,artnr,date,discount,filnr,id,prediction
0,NaN,101,2019-04-23,8,2,446801,NaN
1,NaN,109,2019-04-23,0,2,446802,NaN
2,NaN,110,2019-04-23,0,2,446803,NaN
3,NaN,115,2019-04-23,0,2,446804,NaN
4,NaN,211,2019-04-23,0,2,446805,NaN


In [12]:
len(salesdata[salesdata.filnr==10])

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


50

In [164]:
# for filiale in salesdata.filnr.unique():
#    for article in salesdata.artnr.unique():


In [23]:
preddict = context.catalog.load('preddict')
seo_frame = context.catalog.load('seo_frame')
regressordict = context.catalog.load('regressordict')

2020-12-26 16:16:59,653 - kedro.io.data_catalog - INFO - Loading data from `preddict` (PickleDataSet)...
2020-12-26 16:17:00,780 - kedro.io.data_catalog - INFO - Loading data from `seo_frame` (CSVDataSet)...
2020-12-26 16:17:00,898 - kedro.io.data_catalog - INFO - Loading data from `regressordict` (PickleDataSet)...


In [26]:
postframe

,id,filnr,artnr,date,prediction,base_value,dayspecific,weather,lastmonth,safetystock
0,446801,2,101,2019-04-23,122.0,187,-58,4,-11,114
1,447201,2,101,2019-04-24,95.0,187,-60,5,-36,89
2,447601,2,101,2019-04-25,108.0,187,-61,5,-23,101
3,448001,2,101,2019-04-26,125.0,187,-47,2,-18,117
4,448401,2,101,2019-04-27,186.0,187,7,2,-9,174
...,...,...,...,...,...,...,...,...,...,...
1945,447200,71,626,2019-04-23,20.0,18,3,1,-2,17
1946,447600,71,626,2019-04-24,27.0,18,5,2,2,23
1947,448000,71,626,2019-04-25,23.0,18,4,1,0,20
1948,448400,71,626,2019-04-26,23.0,18,4,0,1,20


In [37]:
import numpy as np
import pandas as pd

In [42]:
np.str(pd.Series(preddict.keys()).unique())

"['2' '3' '4' '5' '14' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26'\n '27' '28' '29' '30' '31' '32' '33' '34' '35' '36' '37' '38' '40' '41'\n '43' '44' '45' '46' '47' '48' '49' '50' '70' '71']"

In [27]:
postframe.prediction.mean()

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


39.48974358974359

In [28]:
postframe.safetystock.mean()

33.02820512820513

In [25]:
postframe = context.catalog.load('postframe')

2020-12-26 16:17:15,410 - kedro.io.data_catalog - INFO - Loading data from `postframe` (CSVDataSet)...


`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


# Lag Data

In [61]:
import asyncio
import aiohttp
import nest_asyncio
nest_asyncio.apply()

/home/lukas/pjs/operativelogik/oplogik/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [46]:
# Input: pred_dates --> start und ende
api_length = 400

In [45]:
pred_dates.iloc[0]#.reset_index()

Timestamp('2019-04-24 00:00:00')

In [40]:
pred_dates.pop(0)

Timestamp('2019-04-23 00:00:00')

In [77]:
ende = str(max(pred_dates) - datetime. timedelta(min(parameters['lags_used'])))[:-9]

In [35]:
start = str(min(pred_dates) - datetime. timedelta(max(parameters['lags_used'])))[:-9]
ende = str(max(pred_dates) - datetime. timedelta(min(parameters['lags_used'])))[:-9]
URL = 'http://35.234.103.119/api/sales/amount?dateFrom=' + start + '&dateTo=' + ende + '&pageSize=' + str(api_length) + '&page='

In [97]:
start = str(min(pred_dates) - datetime. timedelta(max(parameters['lags_used'])))[:-9]
ende = str(max(pred_dates) - datetime. timedelta(min(parameters['lags_used'])))[:-9]
URL = 'http://35.234.103.119/api/sales/amount?dateFrom=' + start + '&dateTo=' + ende + '&pageSize=' + str(api_length) + '&page='
print(URL)

http://35.234.103.119/api/sales/amount?dateFrom=2019-03-27&dateTo=2019-04-23&pageSize=400&page=


/home/lukas/pjs/operativelogik/oplogik/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [102]:
400*28

11200

In [104]:
df_hist['filnr'].unique()

/home/lukas/pjs/operativelogik/oplogik/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([ 2,  3,  4,  5, 10, 14, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
       28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 43, 44, 45, 46,
       47, 48, 49, 50, 70, 71])

In [81]:
def laggetter(pred_dates, api_length):
    start = str(min(pred_dates) - datetime. timedelta(max(parameters['lags_used'])))[:-9]
    ende = str(max(pred_dates) - datetime. timedelta(min(parameters['lags_used'])))[:-9]
    URL = 'http://35.234.103.119/api/sales/amount?dateFrom=' + start + '&dateTo=' + ende + '&pageSize=' + str(api_length) + '&page='
    print(URL)
    pages = range(1, max(parameters['lags_used'])+1)
    list_df = func(URL, pages)
    df = pd.concat(list(pd.Series(list_df).apply(lambda x: pd.json_normalize(x))))    
    df = df.iloc[pd.to_datetime(df.date).values.argsort()].reset_index(drop=True)
    return df

In [48]:
async def fetch(session, url):
    async with session.get(url) as response:
        json_response = await response.json()
        return json_response

async def main(URL, pages):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, (URL+str(i))) for i in pages]
        responses = await asyncio.gather(*tasks)
        return responses

def func(URL, pages):
    responses = asyncio.run(main(URL, pages))
    return responses

In [82]:
df = laggetter(pred_dates, api_length)

http://35.234.103.119/api/sales/amount?dateFrom=2019-03-27&dateTo=2019-04-23&pageSize=400&page=


In [45]:
sales_lag[(sales_lag['filnr']==int(filiale))&(sales_lag['artnr']==int(article))].iloc[-lag:]['amount'].values

/home/lukas/pjs/operativelogik/oplogik/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([ 68,   0, 138, 653, 653, 273,  97])

In [48]:
filfeaturedict[filiale][article]['lag_' + str(lag)] = sales_lag[(sales_lag['filnr']==int(filiale))&(sales_lag['artnr']==int(article))].iloc[-lag:]['amount'].values

/home/lukas/pjs/operativelogik/oplogik/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
filiale = '2'
article = '101'
lag = 7

/home/lukas/pjs/operativelogik/oplogik/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [62]:
lagbase = None
lagbase = -lag
lag = 14

/home/lukas/pjs/operativelogik/oplogik/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [287]:
clipped = sales_lag.drop_duplicates(subset = ['id'])
for filiale in clipped['filnr'].unique():
    for article in clipped['artnr'].unique():
        print(filiale, article, ' ', len(clipped[(clipped['filnr']==int(filiale))&(clipped['artnr']==int(article))]))

10400

In [63]:
sales_lag[(sales_lag['filnr']==int(filiale))&(sales_lag['artnr']==int(article))].iloc[-lag:lagbase]['amount'].values

array([ 90,  72,  68, 114, 306,  61,  95])

In [79]:
start = str(min(pred_dates) - datetime. timedelta(28))[:-9]

In [11]:
'/sales/amount?dateFrom=' + start + '&dateTo=' + ende + '&pageSize=' + str(api_length) + '&page='

/home/lukas/pjs/operativelogik/oplogik/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,amount,artnr,date,filnr,id
0,77,101,2019-03-27,2,436001
1,35,109,2019-03-27,2,436002
2,74,110,2019-03-27,2,436003
3,0,115,2019-03-27,2,436004
4,4,211,2019-03-27,2,436005
...,...,...,...,...,...
11195,0,101,2019-03-30,10,437241
11196,0,109,2019-03-30,10,437242
11197,0,110,2019-03-30,10,437243
11198,0,115,2019-03-30,10,437244


In [2]:
URL = 'http://35.234.103.119/api/sales/amount?dateFrom=%222018-3-20%22&dateTo=%222018-3-22%22'

/home/lukas/pjs/operativelogik/oplogik/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
featureadder(fildict: Dict, filialdata, fdict, weatherdict, parameters: Dict)

In [88]:
basedict = context.catalog.load('basedict')
filialdata = context.catalog.load('filialdata')
fdict = context.catalog.load('fdict')
parameters = context.catalog.load('parameters')
weatherdict = context.catalog.load('weatherdict')

2020-11-17 09:26:04,107 - kedro.io.data_catalog - INFO - Loading data from `basedict` (PickleDataSet)...
2020-11-17 09:26:04,452 - kedro.io.data_catalog - INFO - Loading data from `filialdata` (CSVDataSet)...
2020-11-17 09:26:04,483 - kedro.io.data_catalog - INFO - Loading data from `fdict` (PickleDataSet)...
2020-11-17 09:26:04,514 - kedro.io.data_catalog - INFO - Loading data from `parameters` (MemoryDataSet)...
2020-11-17 09:26:04,540 - kedro.io.data_catalog - INFO - Loading data from `weatherdict` (PickleDataSet)...


In [91]:
basedict['2']['101']

/home/lukas/pjs/operativelogik/oplogik/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,amount,date,discount,id,prediction,wochentag,monat
0,NaN,2019-04-24,8,447201,NaN,2,4
400,NaN,2019-04-25,0,447601,NaN,3,4
800,NaN,2019-04-26,0,448001,NaN,4,4
1200,NaN,2019-04-27,0,448401,NaN,5,4
1600,NaN,2019-04-28,0,448801,NaN,6,4
2000,NaN,2019-04-29,0,449201,NaN,0,4
2400,NaN,2019-04-30,0,449601,NaN,1,4


In [85]:
def featureadder(basedict, filialdata, fdict, weatherdict, parameters, df_hist):
    #filialdata['number_store'] = pd.to_numeric(filialdata['number_store'])
    filialdata.loc[:, 'latlon'] = filialdata.apply(lambda row: str(round(row['lat'], 2)) + ',' + str(round(row['lon'], 2)), axis=1)
    feiertage = fdict['feiertage']
    ferientage = fdict['ferientage']
    for filiale in fildict.keys():
        bundesland = filialdata[filialdata['number_store']==np.float64(filiale)]['state'].values[0]
        vor_feiertag = list(np.array(feiertage[bundesland]) - datetime.timedelta(days=1))
        zweivor_feiertag = list(np.array(feiertage[bundesland]) - datetime.timedelta(days=2))
        fillatlon = filialdata[filialdata['number_store']==int(filiale)]['latlon'].values[0]
        firstart = list(fildict[filiale].keys())[0] # Annahme: Zeitreihen der Filialen sind gleich
        filialweather = weatherdict[fillatlon][weatherdict[fillatlon]['date'].isin(fildict[filiale][firstart]['date'])]
        #Bundesland für jeweilige Filiale
        for artikel in fildict[filiale].keys():
            df = fildict[filiale][artikel]
            df['datecode'] = 0
            df.loc[:, 'date'] = pd.to_datetime(df['date']) #change date to timestamp format
            #Feiertag = 1; Ferien = 2; ein Tag vor Feiertage = 3; zwei Tage vor Feiertag = 4
            df.loc[df['date'].isin(zweivor_feiertag), 'datecode'] = 4
            df.loc[df['date'].isin(vor_feiertag), 'datecode'] = 3
            df.loc[df['date'].isin(ferientage[bundesland]),'datecode'] = 2
            df.loc[df['date'].isin(feiertage[bundesland]),'datecode'] = 1
            #MA and lag features (defiened in parameters)
            #df["ma_" + str(parameters['MA_used'])] = df['x'].rolling(window=parameters['MA_used']).mean()
            for lag in parameters['lags_used']:
                df["lag_" + str(lag)] = df.shift(lag)['amount']
                #df["datelag_" + str(lag)] = df.shift(lag)['datecode']
            df = pd.merge(df, filialweather, how='inner', on=['date', 'date'])
            fildict[filiale][artikel] = df
    return fildict

/home/lukas/pjs/operativelogik/oplogik/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'Dict' is not defined

# Loading the important data

In [46]:
#Problem: brauche eigentlich nur Datums für zukünftige Vorhersage

In [102]:
#locations = 
len(['49.57,9.71', '49.62,9.66', '49.49,9.77', '49.58,9.71',
       '49.63,9.67', '49.51,9.69', '49.48,9.64', '49.52,9.33',
       '49.49,9.78', '49.53,9.33', '49.61,9.74', '49.71,9.26',
       '49.48,9.9', '49.5,9.77', '49.49,9.82', '49.63,9.65', '49.62,9.48',
       '49.54,9.73', '49.57,9.7', '49.37,9.95', '49.78,9.5', '49.8,9.94',
       '49.78,9.93', '49.78,9.88', '49.8,9.93', '49.79,9.91',
       '49.76,9.95', '49.79,9.94', '49.78,9.95', '49.79,9.93',
       '49.43,9.43', '49.61,9.47'])

32

In [89]:
sales[sales['date']==sales[sales['discount']>0]['date'].unique()[0]]

,amount,artnr,date,discount,filnr,id
0,1060,101,2016-01-04,0,2,1
1,10,109,2016-01-04,6,2,2
2,680,110,2016-01-04,0,2,3
3,20,115,2016-01-04,0,2,4
4,0,211,2016-01-04,0,2,5
...,...,...,...,...,...,...
395,70,225,2016-01-04,0,71,396
396,40,231,2016-01-04,0,71,397
397,350,305,2016-01-04,0,71,398
398,0,384,2016-01-04,0,71,399


In [107]:
len(sales['filnr'].unique())

40

In [113]:
for i in range(1, 7):
    print(i, ': ', len(sales[sales['discount']==i]))

1 :  3360
2 :  1120
3 :  14000
4 :  0
5 :  840
6 :  3600


In [100]:
sales['artnr'].unique()

/home/lukas/pjs/operativelogik/oplogik/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([101, 109, 110, 115, 211, 225, 231, 305, 384, 626])

In [78]:
775*len(sales['filnr'].unique())

31000

# API_Abfrage

In [8]:
r = len(requests.get(url='http://35.234.103.119/api/sales/artnr').json())
n_arts = len(r.json())
r = requests.get(url='http://35.234.103.119/api/sales/filnr').json()
n_fils = len(requests.get(url='http://35.234.103.119/api/sales/filnr').json())
#
#total = 'http://35.234.103.119/api/sales/id'

In [38]:
import asyncio
import aiohttp

In [44]:
def framer(pred_duration):
    n_arts = len(requests.get(url='http://35.234.103.119/api/sales/artnr').json())
    n_fils = len(requests.get(url='http://35.234.103.119/api/sales/filnr').json())
    api_length = n_arts * n_fils
    total = 450000
    start = int(total / api_length- pred_duration)
    end = int(total/api_length) + 1
    pages = range(start, end)
    URL = 'http://35.234.103.119/api/sales?&pageSize=' + str(api_length) + '&page='
    list_df = func(URL, pages)
    df = pd.concat(list(pd.Series(list_df).apply(lambda x: pd.json_normalize(x))))
    df = salesdata.append(df)
    return df

In [22]:
predrange = 7

In [42]:
# asynchronen Funktionen für API-Abfrage
async def fetch(session, url):
    async with session.get(url) as response:
        json_response = await response.json()
        return json_response

async def main(URL, pages):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, (URL+str(i))) for i in pages]
        responses = await asyncio.gather(*tasks)
        return responses

def func(URL, pages):
    responses = asyncio.run(main(URL, pages))
    return responses

# Wetterapi

In [ ]:
def weathergetter(locations, parameters, weatherkey): #params:weatherkey
    # Input ps: pd.core.Series mit Datum für historische Vorhersage; z.B: ps = fildict['2']['101']['date']
    # filialdata mit lat und lon-Werten
    filialdata.loc[:, 'latlon'] = filialdata.apply(lambda row: str(round(row['lat'], 2)) + ',' + str(round(row['lon'], 2)), axis=1)
    weatherdict = {}
    apifails = {}
    ps = pd.to_datetime(pd.Series(parameters['pred_dates']))
    for location in filialdata['latlon'].unique():
        queries = daynator(ps)
        requestdata = weatherrequester(ps, queries, location, apifails, weatherkey)
        #nur Datum bis heute über historische Vorhersage-API
        latlonweather = requestdata[0]
        #apifails = requestdata[1]
        weatherdict[location] = pd.DataFrame.from_dict(latlonweather)
    return [weatherdict, apifails]

In [56]:
for date in parameters['pred_dates']:
    print(date[:-9])

2019-04-24
2019-04-25
2019-04-26
2019-04-27
2019-04-28
2019-04-29
2019-04-30


In [8]:
def daynator(ps:pd.core.series.Series):
    first_days = []
    last_days = []
    querydict = {}
    for year in ps.apply(lambda x: x.year).unique():
        for month in ps[ps.apply(lambda x: x.year)==year].apply(lambda x: x.month).unique():
            first_day_int = min(ps[(ps.apply(lambda x: x.year)==year) & (ps.apply(lambda x: x.month)==month)].apply(lambda x: x.day))
            first_days.append(str(year)+'-'+str(month).zfill(2)+'-'+ str(first_day_int))
            last_day_int = max(ps[(ps.apply(lambda x: x.year)==year) & (ps.apply(lambda x: x.month)==month)].apply(lambda x: x.day))
            last_days.append(str(year)+'-'+str(month).zfill(2)+'-'+ str(last_day_int))
    querydict['first_days'] = pd.to_datetime(first_days)
    querydict['last_days'] = pd.to_datetime(last_days)
    return pd.DataFrame.from_dict(querydict)

In [26]:
ps = pd.to_datetime(pd.Series(parameters['pred_dates']))
queries = daynator(ps)

/home/lukas/pjs/operativelogik/oplogik/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [52]:
parameters['pred_dates']

['2019-04-24 00:00:00',
 '2019-04-25 00:00:00',
 '2019-04-26 00:00:00',
 '2019-04-27 00:00:00',
 '2019-04-28 00:00:00',
 '2019-04-29 00:00:00',
 '2019-04-30 00:00:00']

In [62]:
example1 = ['2020-11-06 00:00:00',
 '2020-11-09 00:00:00',
 '2020-11-10 00:00:00',
 '2020-11-11 00:00:00',
 '2020-11-12 00:00:00',
 '2020-11-13 00:00:00',]

In [63]:
queries = daynator(pd.to_datetime(pd.Series(example1)))

In [49]:
queries['first_days'][0]

Timestamp('2019-04-24 00:00:00')

In [50]:
datetime.date.today()

datetime.date(2020, 11, 7)

In [51]:
queries['last_days'][0]

Timestamp('2019-04-30 00:00:00')

In [60]:
str(datetime.date.today())

'2020-11-07'

In [70]:
duration = str((queries['last_days'][0]- pd.Timestamp(datetime.date.today())).days +1)

In [71]:
duration

'6'

In [64]:
if datetime.date.today() >= queries['last_days'][i]:
    print(1)
    #historisch
elif datetime.date.today() < queries['first_days'][i]:
    print(2)
    #zukünftig: Input: 
else:
    #erste fkt Input: queries['first_days'][0], datetime.date.today()
    #zweite fkt: Input: 
    print(3)
    
#split: -today & ab morgen

3


/home/lukas/pjs/operativelogik/oplogik/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# wetter checken: 
# ps[ps<=pd.to_datetime(datetime.date.today())]
# ps[ps>pd.to_datetime(datetime.date.today())]

In [85]:
datadict = {'date': np.empty(0, dtype='datetime64[ns]'), 'sunhour':np.empty(0),
 'feels_like':np.empty(0), 'heat_index':np.empty(0), 'weathercodes': np.empty(0)}

In [114]:
queries

,first_days,last_days
0,2020-11-06,2020-11-13


In [149]:
new = weatherrequester(queries, location, {}, weatherkey)

In [151]:
new[0].keys()

dict_keys(['date', 'sunhour', 'weathercodes', 'feels_like', 'heat_index'])

In [128]:
datadict = {'date': np.empty(0, dtype='datetime64[ns]')}

In [131]:
datadict['date']

array([], dtype='datetime64[ns]')

In [136]:
datadict['date'] = np.append(datadict['date'], pd.to_datetime(jsonframe['date']).values)

In [137]:
datadict['date']

array(['2020-11-09T00:00:00.000000000', '2020-11-10T00:00:00.000000000',
       '2020-11-11T00:00:00.000000000', '2020-11-12T00:00:00.000000000',
       '2020-11-13T00:00:00.000000000', '2020-11-14T00:00:00.000000000',
       '2020-11-15T00:00:00.000000000', '2020-11-09T00:00:00.000000000',
       '2020-11-10T00:00:00.000000000', '2020-11-11T00:00:00.000000000',
       '2020-11-12T00:00:00.000000000', '2020-11-13T00:00:00.000000000',
       '2020-11-14T00:00:00.000000000', '2020-11-15T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [146]:
def weatherrequester(queries, location, apifails, weatherkey):
    latlonweather = {}
    datadict = {'date': np.empty(0, dtype='datetime64[ns]'), 'sunhour':np.empty(0),
     'feels_like':np.empty(0), 'heat_index':np.empty(0), 'weathercodes': np.empty(0)}
    # --> hier ggf. auf queryfunktion
    for i in range(0, len(queries)):
        if datetime.date.today() >= queries['last_days'][i]:
            # nur historisch: gleiche wie in planungslogik
            start_date = str(queries['first_days'][i])[:-9]
            end_date = str(queries['last_days'][i])[:-9]
            [datadict, apifails] = wheaterhist(apifails, datadict, weatherkey, location, start_date, end_date)
            #historisch
        elif datetime.date.today() < queries['first_days'][i]:
            #nur zukünftig: duration = Zeitunterschied zu heute
            duration = str((queries['last_days'][i]- pd.Timestamp(datetime.date.today())).days)
            [datadict, apifails] = wheatherpred(apifails, datadict, weatherkey, location, duration)
        else:
            # historisch + zukünftig -> zuerst hist, dann pred
            start_date = str(queries['first_days'][i])[:-9]
            end_date = str(datetime.date.today())
            [datadict, apifails] = wheaterhist(apifails, datadict, weatherkey, location, start_date, end_date)
            duration = str((queries['last_days'][i]- pd.Timestamp(datetime.date.today())).days)
            [datadict, apifails] = wheatherpred(apifails, datadict, weatherkey, location, duration)
    latlonweather['date'] = datadict['date']
    latlonweather['sunhour'] = datadict['sunhour']
    latlonweather['weathercodes'] = datadict['weathercodes']
    latlonweather['feels_like'] = datadict['feels_like']
    latlonweather['heat_index'] = datadict['heat_index']
    return [latlonweather, apifails]

In [145]:
def wheaterhist(apifails, datadict, weatherkey, location, start_date, end_date):
    URL = 'http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=' + weatherkey + '&q=' + location + '&date=' + start_date + '&enddate=' + end_date + '&format=json&tp=24'
    datadict = apicrawler(apifails, URL, datadict)
    return [datadict, apifails]

In [144]:
def wheatherpred(apifails, datadict, weatherkey, location, duration):
    URL = ('https://api.worldweatheronline.com/premium/v1/weather.ashx?key=' +
           weatherkey + '&q=' + location + '&num_of_days=' + duration + '&cc=no' + '&format=json&tp=24')
    datadict = apicrawler(apifails, URL, datadict) 
    return [datadict, apifails]

In [148]:
def apicrawler(apifails, URL, datadict):
    r = requests.get(url=URL)
    while True: #nötig, falls request failed
        try:
            data = r.json()['data']['weather']
            break
        except:
            apifails[location] = [str(datetime.date.today()), duration]
            r = requests.get(url=URL)
            data = r.json()['data']['weather']
            break
    jsonframe = pd.DataFrame.from_dict(data)
    datadict['date'] = np.append(datadict['date'], pd.to_datetime(jsonframe['date']).values)
    datadict['sunhour'] = np.append(datadict['sunhour'], np.float64(jsonframe['sunHour'].values))
    datadict['weathercodes'] = np.append(datadict['weathercodes'],np.float64(jsonframe.apply(lambda row: row['hourly'][0]['weatherCode'], axis=1).values))
    datadict['feels_like'] = np.append(datadict['feels_like'],np.float64(jsonframe.apply(lambda row: row['hourly'][0]['FeelsLikeC'], axis=1).values))
    datadict['heat_index'] = np.append(datadict['heat_index'], np.float64(jsonframe.apply(lambda row: row['hourly'][0]['HeatIndexC'], axis=1).values))
    return datadict

In [92]:
r = requests.get(url=URL)
while True: #nötig, falls request failed
    try:
        data = r.json()['data']['weather']
        break
    except:
        apifails[location] = [str(datetime.date.today()), duration]
        r = requests.get(url=URL)
        data = r.json()['data']['weather']
        break

In [94]:
jsonframe = pd.DataFrame.from_dict(data)

In [119]:
weatherkey = 'f829e43bc7bd465f8db92239201910'
location = '49.57,9.71'
duration = '7'

In [108]:
new = wheatherpred({}, datadict, weatherkey, location, duration)

In [111]:
[new, life] = wheatherpred({}, datadict, weatherkey, location, duration)

In [96]:
pd.to_datetime(jsonframe['date']).values

/home/lukas/pjs/operativelogik/oplogik/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['2020-11-09T00:00:00.000000000', '2020-11-10T00:00:00.000000000',
       '2020-11-11T00:00:00.000000000', '2020-11-12T00:00:00.000000000',
       '2020-11-13T00:00:00.000000000', '2020-11-14T00:00:00.000000000',
       '2020-11-15T00:00:00.000000000'], dtype='datetime64[ns]')

In [37]:
('https://api.worldweatheronline.com/premium/v1/weather.ashx ' + '?key='
 +'f829e43bc7bd465f8db92239201910'+ '&q=' + '49.57,9.71' +
 '&num_of_days=' + duration + '&cc=no' + '&format=json&tp=24')

'https://api.worldweatheronline.com/premium/v1/weather.ashx ?key=f829e43bc7bd465f8db92239201910&q=49.57,9.71&date=2020-11-10&num_of_days=6&cc=no&format=json&tp=24'

In [ ]:
http://api.worldweatheronline.com/premium/v1/past-weather.ashx/?key=f829e43bc7bd465f8db92239201910&q=49.57,9.71&date=2020-11-01&enddate=2020-11-07&format=json&tp=24

# Archive

In [ ]:
'''
def salesgenerator(parameters):
    df = pd.DataFrame(columns = ['amount', 'artnr', 'date', 'discount', 'filnr', 'id'])
    i = 1
    while True:
        try:
            URL = "https://level3-microservice.ew.r.appspot.com/api/employee/all/" + str(i)
            r = requests.get(url=URL)
            appender = pd.DataFrame(r.json())
            appender['date'].head(1)
            df = df.append(appender)
            i += 1
        except:
            print(i)
            break
    df['date'] = pd.to_datetime(df['date'])
    df = df[df['date'].isin(parameters['pred_dates']) &  
            df['artnr'].isin(parameters['artikel']) &
                df['filnr'].isin(parameters['filialen'])]
    df.drop('id', axis = 1, inplace = True)
    return df
'''